In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline

def compute_standard_wCD(N):
    # extracted from .cf2DistFFT.py
    xMin = 0
    xMax = N
    xRange = xMax - xMin
    dt  = 2*np.pi / xRange
    # dt = 1/xRange
    k   = np.arange(N, dtype=complex)     # np.complex is deprecated, or use np.complex128
    w   = (k - N/2 + 0.5) * dt
    A   = xMin
    B   = xMax
    # dx  = (B-A)/N
    c   = (-1)**(A*(N-1)/(B-A))/(B-A)
    # print("A, B, N, dx, c=", A, B, N, dx, c)
    C = c * (-1)**((1-1/N)*k)
    D = (-1)**(-2*(A/(B-A))*k)     # k must be complex, see https://stackoverflow.com/questions/45384602/numpy-runtimewarning-invalid-value-encountered-in-power
    return w, C, D

class FftInvPdf:
    def __init__(self, cf):
        self.cf = cf
        self.N = N = 1024
        self.w, self.C, self.D = compute_standard_wCD(N)

    def __call__(self, t, *params):
        N = self.N
        cft = self.cf(self.w[N//2:], *params)
        cft = np.concatenate([cft[::-1].conj(), cft])
        pdfFFT = np.max([np.zeros(N), (self.C*np.fft.fft(self.D*cft)).real], axis=0)
        spline = UnivariateSpline(np.arange(N), pdfFFT, s=0)
        return spline(t)
from scipy.interpolate import UnivariateSpline

def compute_standard_wCD(N):
    # extracted from .cf2DistFFT.py
    xMin = 0
    xMax = N
    xRange = xMax - xMin
    dt  = 2*np.pi / xRange
    # dt = 1/xRange
    k   = np.arange(N, dtype=complex)     # np.complex is deprecated, or use np.complex128
    w   = (k - N/2 + 0.5) * dt
    A   = xMin
    B   = xMax
    # dx  = (B-A)/N
    c   = (-1)**(A*(N-1)/(B-A))/(B-A)
    # print("A, B, N, dx, c=", A, B, N, dx, c)
    C = c * (-1)**((1-1/N)*k)
    D = (-1)**(-2*(A/(B-A))*k)     # k must be complex, see https://stackoverflow.com/questions/45384602/numpy-runtimewarning-invalid-value-encountered-in-power
    return w, C, D

class FftInvPdf:
    def __init__(self, cf):
        self.cf = cf
        self.N = N = 1024
        self.w, self.C, self.D = compute_standard_wCD(N)

    def __call__(self, t, *params):
        N = self.N
        cft = self.cf(self.w[N//2:], *params)
        cft = np.concatenate([cft[::-1].conj(), cft])
        pdfFFT = np.max([np.zeros(N), (self.C*np.fft.fft(self.D*cft)).real], axis=0)
        spline = UnivariateSpline(np.arange(N), pdfFFT, s=0)
        return spline(t)

In [ ]:
def dispersive_monopore(w, npi, tpi, N0, t0):
    Z = npi*(1/(1 - 1j*w*tpi) - 1) + 1j*w*t0
    return np.exp(Z + Z**2/(2*N0))

dispersive_monopore_pdf_impl = FftInvPdf(dispersive_monopore)

DEFUALT_TIMESCALE = 0.25    # 0.1 for FER_OA
N0 = 14400.0    # 48000*0.3 (30cm) or (t0/σ0)**2, see meeting document 20221104/index.html 

def dispersive_monopore_pdf(x, npi, tpi, N0, t0, timescale=DEFUALT_TIMESCALE):
    return timescale*dispersive_monopore_pdf_impl(timescale*x, npi, timescale*tpi, N0, timescale*t0)


## Gamma Residence Time Model

Extend to **Gamma-distributed** residence times instead of exponential.

**Gamma distribution parameters:**
- `k` (shape): Controls distribution shape
  - `k = 1`: Recovers exponential (memoryless)
  - `k > 1`: More concentrated, less skewed than exponential
  - `k < 1`: Heavy-tailed, longer residence times
- `theta` (scale): Mean residence time per pore = `k * theta`

**Physical interpretation:**
- `k` represents number of independent sub-processes
- Heterogeneous pore surface → multiple binding sites → Gamma-like behavior

In [ ]:
def dispersive_gamma_pore(w, npi, k, theta, N0, t0):
    """
    Gamma-distributed residence times with mobile phase dispersion.
    
    Parameters:
    -----------
    w : array
        Frequency array
    npi : float
        Mean number of pore entries (Poisson parameter)
    k : float
        Gamma shape parameter (k=1 recovers exponential)
    theta : float
        Gamma scale parameter (mean residence = k*theta)
    N0 : float
        Plate number (mobile phase dispersion)
    t0 : float
        Mean mobile phase time
    
    Returns:
    --------
    CF : complex array
        Characteristic function
        
    Notes:
    ------
    - Exponential: CF = 1/(1 - iω*τ)
    - Gamma: CF = (1 - iω*θ)^(-k)
    - For k=1, θ=τ: Gamma → Exponential
    """
    # Gamma CF for single pore visit: (1 - iω*θ)^(-k)
    # CPP with Gamma jumps: exp[n*(CF - 1)]
    Z = npi*((1 - 1j*w*theta)**(-k) - 1) + 1j*w*t0
    return np.exp(Z + Z**2/(2*N0))

# Create PDF calculator
dispersive_gamma_pore_pdf_impl = FftInvPdf(dispersive_gamma_pore)

def dispersive_gamma_pore_pdf(x, npi, k, theta, N0, t0, timescale=DEFUALT_TIMESCALE):
    """Wrapper with timescale normalization"""
    return timescale*dispersive_gamma_pore_pdf_impl(
        timescale*x, npi, k, timescale*theta, N0, timescale*t0
    )

### Comparison: Exponential vs Gamma Residence Times

Compare different `k` values:
- `k = 0.5`: Heavy-tailed (some molecules trapped long time)
- `k = 1.0`: Exponential (baseline, memoryless)
- `k = 2.0`: More concentrated, less variable
- `k = 5.0`: Nearly deterministic residence times

In [ ]:
# Parameters
t = np.linspace(0.01, 300, 200)
np_ = 100
mean_residence = 1.0  # Keep mean residence time constant
t0_val = mean_residence / 2

# For Gamma with mean = k*theta, we set theta = mean/k
k_values = [0.5, 1.0, 2.0, 5.0]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, k in enumerate(k_values):
    ax = axes[idx]
    theta = mean_residence / k  # Maintain constant mean
    
    # Calculate PDFs
    pdf_gamma = dispersive_gamma_pore_pdf(t, np_, k, theta, N0, t0_val)
    pdf_exp = dispersive_monopore_pdf(t, np_, mean_residence, N0, t0_val)
    
    # Plot
    ax.plot(t, pdf_exp, 'k--', label='Exponential (k=1)', linewidth=2, alpha=0.7)
    ax.plot(t, pdf_gamma, 'r-', label=f'Gamma (k={k})', linewidth=2)
    
    # Statistics
    # Mean should be approximately the same: npi*mean_residence + t0
    theoretical_mean = np_ * mean_residence + t0_val
    
    ax.axvline(theoretical_mean, color='blue', linestyle=':', alpha=0.5, label=f'Theory mean={theoretical_mean:.1f}')
    ax.set_xlabel('Time', fontsize=11)
    ax.set_ylabel('PDF', fontsize=11)
    ax.set_title(f'Shape k = {k}' + (' (exponential)' if k==1.0 else ''), fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    
    # Add text annotation about shape
    if k < 1:
        ax.text(0.6, 0.95, 'Heavy tail\n(trapped molecules)', 
                transform=ax.transAxes, fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    elif k > 1:
        ax.text(0.6, 0.95, 'More concentrated\n(uniform escape)', 
                transform=ax.transAxes, fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

plt.tight_layout()
plt.suptitle('Effect of Gamma Shape Parameter on Residence Time Distribution', 
             fontsize=14, fontweight='bold', y=1.002)
plt.show()

# Quantitative comparison
print("Quantitative Comparison (all with same mean residence time = {:.1f})".format(mean_residence))
print("="*70)
print(f"{'k value':<10} {'Peak height':<15} {'Peak position':<15} {'Skewness':<15}")
print("-"*70)

for k in k_values:
    theta = mean_residence / k
    pdf_gamma = dispersive_gamma_pore_pdf(t, np_, k, theta, N0, t0_val)
    peak_idx = np.argmax(pdf_gamma)
    peak_height = pdf_gamma[peak_idx]
    peak_pos = t[peak_idx]
    
    # Approximate skewness from moments
    m1 = np.trapz(t * pdf_gamma, t)
    m2 = np.trapz((t - m1)**2 * pdf_gamma, t)
    m3 = np.trapz((t - m1)**3 * pdf_gamma, t)
    skew = m3 / m2**(3/2) if m2 > 0 else 0
    
    print(f"{k:<10.1f} {peak_height:<15.6f} {peak_pos:<15.2f} {skew:<15.3f}")


### Analytical Moments for Gamma Model

The Lévy framework allows us to compute moments analytically from cumulants:

**For Exponential (k=1):**
- Mean: μ = n̄p·τ̄ + t₀
- Variance: σ² = 2n̄p·τ̄² + t₀²/N₀

**For Gamma (shape k):**
- Mean: μ = n̄p·k·θ + t₀ (same if k·θ = τ̄)
- Variance: σ² = 2n̄p·k·θ² + t₀²/N₀

Key insight: **Variance scales with k** for fixed mean!

In [ ]:
def compute_analytical_moments_gamma(npi, k, theta, N0, t0):
    """
    Compute analytical moments for Gamma residence time model.
    
    Returns:
    --------
    dict with keys: mean, variance, std, cv (coefficient of variation)
    """
    # First moment (mean)
    mean = npi * k * theta + t0
    
    # Second moment about origin
    # For Gamma CPP: E[X²] = n*(k*θ)² + n*k*θ²
    # Plus dispersion term from subordination
    variance_stationary = 2 * npi * k * theta**2  # From CPP + Gamma
    variance_mobile = t0**2 / N0  # From mobile dispersion
    variance = variance_stationary + variance_mobile
    
    std = np.sqrt(variance)
    cv = std / mean if mean > 0 else 0  # Coefficient of variation
    
    return {
        'mean': mean,
        'variance': variance,
        'std': std,
        'cv': cv,
        'variance_stationary': variance_stationary,
        'variance_mobile': variance_mobile
    }

# Compare analytical vs numerical moments
print("Analytical Moments Validation")
print("="*80)
print(f"{'k':<6} {'Mean (theory)':<15} {'Mean (numerical)':<18} {'Std (theory)':<15} {'Std (numerical)':<15}")
print("-"*80)

for k in [0.5, 1.0, 2.0, 5.0]:
    theta = mean_residence / k  # Keep mean constant
    
    # Analytical
    moments = compute_analytical_moments_gamma(np_, k, theta, N0, t0_val)
    
    # Numerical from PDF
    pdf_gamma = dispersive_gamma_pore_pdf(t, np_, k, theta, N0, t0_val)
    mean_num = np.trapz(t * pdf_gamma, t)
    var_num = np.trapz((t - mean_num)**2 * pdf_gamma, t)
    std_num = np.sqrt(var_num)
    
    print(f"{k:<6.1f} {moments['mean']:<15.2f} {mean_num:<18.2f} {moments['std']:<15.2f} {std_num:<15.2f}")

print("\n" + "="*80)
print("Variance Decomposition for k=1 (Exponential case)")
print("-"*80)
k = 1.0
theta = mean_residence / k
moments = compute_analytical_moments_gamma(np_, k, theta, N0, t0_val)
print(f"Total variance:       {moments['variance']:.2f}")
print(f"Stationary phase:     {moments['variance_stationary']:.2f} ({100*moments['variance_stationary']/moments['variance']:.1f}%)")
print(f"Mobile phase:         {moments['variance_mobile']:.2f} ({100*moments['variance_mobile']/moments['variance']:.1f}%)")
print(f"Coefficient of var:   {moments['cv']:.4f}")

### Key Observations

**1. Model Flexibility:**
- Gamma model has one extra parameter (`k`) compared to exponential
- When `k = 1`, Gamma → Exponential (nested models)
- Can test if `k ≠ 1` improves fit to data

**2. Physical Interpretation:**
- `k < 1`: Heavy-tailed → some molecules get trapped for long times (heterogeneous escape barriers)
- `k = 1`: Memoryless → standard exponential (homogeneous Brownian escape)
- `k > 1`: More uniform → tighter distribution of residence times (multiple independent sub-processes)

**3. Peak Shape Changes:**
- Smaller `k` → broader peaks, longer tails
- Larger `k` → sharper peaks, more symmetric
- All maintain same retention time if mean is constant

**4. Identifiability Question:**
- Can we distinguish `k ≠ 1` from just adjusting `N0`?
- Need experimental data to test!
- Statistical test: Likelihood ratio test (Gamma vs Exponential)

In [ ]:
from scipy.special import iv, ive

def gec_monopore_pdf(t, np_, tp_):
    return iv(1, np.sqrt(4*np_*t/tp_)) * np.sqrt(np_/(t*tp_)) * np.exp(-t/tp_-np_)

def robust_gec_monopore_pdf(t, np_, tp_):
    # Bessel functions in Python that work with large exponents
    # https://stackoverflow.com/questions/13726464/bessel-functions-in-python-that-work-with-large-exponents
    #
    # iv(1, np.sqrt(4*np_*t/tp_)) * np.sqrt(np_/(t*tp_)) * np.exp(-t/tp_-np_)
    #
    # ive(v, z) = iv(v, z) * exp(-abs(z.real))
    # iv(v, sq) = ive(v, sq) * exp(sq)

    # val = single_pore_pdf(t, np_, tp_)
    sq = np.sqrt(4*np_*t/tp_)
    val = ive(1, sq) * np.sqrt(np_/(t*tp_)) * np.exp(sq -t/tp_ -np_)
    isnan_val = np.isnan(val)
    val[isnan_val] = 0
    return val

from molass_legacy.SecTheory.SecPDF import FftInvPdf
def gec_monopore_cf(s, np_, tp_):
    # Characteristic function of the GEC monopore model
    return np.exp(np_ * (1/(1 - 1j * tp_ * s) - 1))
 
gec_monopore_numerical_inversion_pdf = FftInvPdf(gec_monopore_cf)

In [ ]:
t = np.linspace(0.01, 300, 100)
np_ = 100
tp_ = 1
pdf1 = gec_monopore_pdf(t, np_, tp_)
pdf2 = robust_gec_monopore_pdf(t, np_, tp_)
pdf3 = gec_monopore_numerical_inversion_pdf(t, np_, tp_)
pfd4  = dispersive_monopore_pdf(t, np_, tp_, N0, tp_/2)
plt.plot(t, pdf1, label='gec_monopore_pdf') 
plt.plot(t, pdf2, label='robust_gec_monopore_pdf', linestyle='dashed')
plt.plot(t, pdf3, label='gec_monopore_numerical_inversion_pdf', linestyle='dotted')
plt.plot(t, pfd4, label='dispersive_monopore_pdf', linestyle='dashdot')
plt.legend()

## Explicit Subordination Structure

### Current Implementation: Implicit Subordination (Dondi)

The dispersive model uses **Bochner subordination** implicitly:
```python
Z = npi*(1/(1 - 1j*w*tpi) - 1) + 1j*w*t0
φ(ω) = exp(Z + Z²/(2*N0))
```

The `Z²/(2*N0)` term encodes subordination, but **two timescales are mixed** in one formula.

---

### Proposed: Explicit Subordination

**Separate the subordinator from the subordinated process:**

1. **Primary process (Stationary phase):** CPP with Gamma jumps
   - Characteristic exponent: `Ψ_stat(ω) = npi*((1 - iω*θ)^(-k) - 1)`
   - Physical meaning: Random pore residence times

2. **Subordinator (Mobile phase dispersion):** Gaussian operational time
   - `T ~ N(t₀, t₀²/N₀)`
   - Physical meaning: Random observation time due to dispersion

3. **Subordinated process:** Evaluate stationary phase at random time T
   - CF: `φ(ω) = E[exp(iω·X(T))]`
   - Requires: Evaluate CPP at random Gaussian time

---

### Mathematical Framework

For Lévy subordination with Gaussian subordinator:

If `T ~ N(t₀, t₀²/N₀)` and `X(t)` has characteristic exponent `Ψ_X(ω)`, then:
$$\phi_Y(\omega) = E[\exp(i\omega \cdot X(T))] = \exp\left(t_0 \Psi_X(\omega) + \frac{t_0^2}{2N_0} \Psi_X(\omega)^2\right)$$

This is because:
$$E[\exp(i\omega \cdot X(T))] = E[\exp(T \cdot \Psi_X(\omega))] = \exp\left(t_0 \Psi_X(\omega) + \frac{t_0^2}{2N_0} \Psi_X(\omega)^2\right)$$

where the last step uses the Gaussian MGF: `E[exp(λT)] = exp(λμ + λ²σ²/2)`

In [ ]:
def explicit_subordination_cf(w, npi, k, theta, N0, t0):
    """
    Explicit subordination: Gaussian subordinator applied to CPP-Gamma process.
    
    Structure:
    ---------
    1. Primary (stationary): Ψ_X(ω) = npi*((1 - iω*θ)^(-k) - 1)
    2. Subordinator (mobile): T ~ N(t₀, t₀²/N₀)
    3. Result: φ(ω) = exp(t₀*Ψ_X(ω) + (t₀²/(2N₀))*Ψ_X(ω)²)
    
    Mathematical derivation:
    -----------------------
    For subordination with Gaussian T:
      E[exp(iω·X(T))] = E[exp(T·Ψ_X(ω))]
                      = exp(μ_T·Ψ_X(ω) + (σ_T²/2)·Ψ_X(ω)²)
    
    where μ_T = t₀, σ_T² = t₀²/N₀
    
    Returns:
    --------
    CF : complex array
        Characteristic function of subordinated process
    """
    # Step 1: Characteristic exponent of stationary phase (CPP-Gamma)
    Psi_X = npi * ((1 - 1j*w*theta)**(-k) - 1)
    
    # Step 2: Apply Gaussian subordination
    # E[exp(iω·X(T))] where T ~ N(t₀, t₀²/N₀)
    phi = np.exp(t0 * Psi_X + (t0**2 / (2*N0)) * Psi_X**2)
    
    return phi

# Create PDF calculator
explicit_subordination_pdf_impl = FftInvPdf(explicit_subordination_cf)

def explicit_subordination_pdf(x, npi, k, theta, N0, t0, timescale=DEFUALT_TIMESCALE):
    """Wrapper with timescale normalization"""
    return timescale*explicit_subordination_pdf_impl(
        timescale*x, npi, k, timescale*theta, N0, timescale*t0
    )

### Comparison: Dondi vs Explicit Subordination

**Key question:** Are they mathematically equivalent?

**Dondi formula (implicit):**
```python
Z = npi*((1 - iω*θ)^(-k) - 1) + iω*t₀
φ_Dondi(ω) = exp(Z + Z²/(2*N₀))
```

**Explicit subordination:**
```python
Ψ_X(ω) = npi*((1 - iω*θ)^(-k) - 1)
φ_Explicit(ω) = exp(t₀*Ψ_X(ω) + (t₀²/(2N₀))*Ψ_X(ω)²)
```

**Are they the same?**

Let's expand explicit subordination:
```
φ_Explicit = exp(t₀*[npi*((1-iω*θ)^(-k)-1)] + (t₀²/(2N₀))*[npi*((1-iω*θ)^(-k)-1)]²)
```

For Dondi:
```
Z = npi*((1-iω*θ)^(-k)-1) + iω*t₀
Z² = [npi*((1-iω*θ)^(-k)-1)]² + 2*iω*t₀*npi*((1-iω*θ)^(-k)-1) + (iω*t₀)²

φ_Dondi = exp(Z + Z²/(2N₀))
        = exp(npi*((1-iω*θ)^(-k)-1) + iω*t₀ + Z²/(2N₀))
```

**They are NOT the same!** The key difference:
- Dondi has **cross term**: `2*iω*t₀*npi*((1-iω*θ)^(-k)-1)` in Z²
- Explicit lacks this cross term

This suggests **different physical interpretations**!

In [ ]:
# Numerical comparison: Dondi vs Explicit Subordination
t = np.linspace(0.01, 300, 200)
np_ = 100
k_test = 2.0  # Use k=2 (from empirical validation showing k>1)
mean_residence = 1.0
theta_test = mean_residence / k_test
t0_val = mean_residence / 2

# Calculate PDFs
pdf_dondi = dispersive_gamma_pore_pdf(t, np_, k_test, theta_test, N0, t0_val)
pdf_explicit = explicit_subordination_pdf(t, np_, k_test, theta_test, N0, t0_val)

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Panel 1: Overlay
ax = axes[0, 0]
ax.plot(t, pdf_dondi, 'b-', label='Dondi (implicit)', linewidth=2)
ax.plot(t, pdf_explicit, 'r--', label='Explicit subordination', linewidth=2, alpha=0.7)
ax.set_xlabel('Time', fontsize=11)
ax.set_ylabel('PDF', fontsize=11)
ax.set_title('PDF Comparison (k=2.0)', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Panel 2: Difference
ax = axes[0, 1]
difference = pdf_dondi - pdf_explicit
ax.plot(t, difference, 'purple', linewidth=2)
ax.axhline(0, color='black', linestyle=':', alpha=0.5)
ax.set_xlabel('Time', fontsize=11)
ax.set_ylabel('Difference (Dondi - Explicit)', fontsize=11)
ax.set_title('Difference Between Methods', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Panel 3: Relative difference (%)
ax = axes[1, 0]
rel_diff = 100 * (pdf_dondi - pdf_explicit) / (pdf_dondi + 1e-10)
ax.plot(t, rel_diff, 'orange', linewidth=2)
ax.axhline(0, color='black', linestyle=':', alpha=0.5)
ax.set_xlabel('Time', fontsize=11)
ax.set_ylabel('Relative Difference (%)', fontsize=11)
ax.set_title('Relative Difference', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

# Panel 4: Log scale overlay
ax = axes[1, 1]
ax.semilogy(t, pdf_dondi, 'b-', label='Dondi (implicit)', linewidth=2)
ax.semilogy(t, pdf_explicit, 'r--', label='Explicit subordination', linewidth=2, alpha=0.7)
ax.set_xlabel('Time', fontsize=11)
ax.set_ylabel('PDF (log scale)', fontsize=11)
ax.set_title('Log Scale (shows tail behavior)', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Dondi vs Explicit Subordination (k=2.0)', fontsize=14, fontweight='bold', y=1.002)
plt.show()

# Quantitative metrics
print("="*80)
print("QUANTITATIVE COMPARISON: Dondi vs Explicit Subordination")
print("="*80)
print(f"Parameters: np={np_}, k={k_test}, θ={theta_test:.3f}, N₀={N0}, t₀={t0_val}")
print("-"*80)

# Moments
mean_dondi = np.trapz(t * pdf_dondi, t)
mean_explicit = np.trapz(t * pdf_explicit, t)
var_dondi = np.trapz((t - mean_dondi)**2 * pdf_dondi, t)
var_explicit = np.trapz((t - mean_explicit)**2 * pdf_explicit, t)

print(f"Mean (Dondi):     {mean_dondi:.4f}")
print(f"Mean (Explicit):  {mean_explicit:.4f}")
print(f"Mean difference:  {abs(mean_dondi - mean_explicit):.6f} ({100*abs(mean_dondi-mean_explicit)/mean_dondi:.3f}%)")
print()
print(f"Std (Dondi):      {np.sqrt(var_dondi):.4f}")
print(f"Std (Explicit):   {np.sqrt(var_explicit):.4f}")
print(f"Std difference:   {abs(np.sqrt(var_dondi) - np.sqrt(var_explicit)):.6f}")
print()

# Peak characteristics
peak_idx_dondi = np.argmax(pdf_dondi)
peak_idx_explicit = np.argmax(pdf_explicit)
print(f"Peak position (Dondi):     {t[peak_idx_dondi]:.4f}")
print(f"Peak position (Explicit):  {t[peak_idx_explicit]:.4f}")
print(f"Peak height (Dondi):       {pdf_dondi[peak_idx_dondi]:.6f}")
print(f"Peak height (Explicit):    {pdf_explicit[peak_idx_explicit]:.6f}")
print()

# Integral check (should be ~1)
integral_dondi = np.trapz(pdf_dondi, t)
integral_explicit = np.trapz(pdf_explicit, t)
print(f"Integral (Dondi):    {integral_dondi:.6f}")
print(f"Integral (Explicit): {integral_explicit:.6f}")
print()

# Maximum absolute difference
max_abs_diff = np.max(np.abs(pdf_dondi - pdf_explicit))
max_rel_diff = np.max(np.abs(rel_diff[np.isfinite(rel_diff)]))
print(f"Max absolute difference: {max_abs_diff:.6f}")
print(f"Max relative difference: {max_rel_diff:.2f}%")
print("="*80)

### Test Across Different k Values

Let's see how the difference between Dondi and Explicit subordination changes with shape parameter k:

In [ ]:
# Compare across k values
k_values = [0.5, 1.0, 2.0, 5.0]
t = np.linspace(0.01, 300, 200)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

results_table = []

for idx, k in enumerate(k_values):
    ax = axes[idx]
    theta = mean_residence / k
    
    # Calculate PDFs
    pdf_dondi = dispersive_gamma_pore_pdf(t, np_, k, theta, N0, t0_val)
    pdf_explicit = explicit_subordination_pdf(t, np_, k, theta, N0, t0_val)
    
    # Plot
    ax.plot(t, pdf_dondi, 'b-', label='Dondi', linewidth=2)
    ax.plot(t, pdf_explicit, 'r--', label='Explicit', linewidth=2, alpha=0.7)
    
    # Difference (shaded)
    ax.fill_between(t, pdf_dondi, pdf_explicit, alpha=0.3, color='purple', label='Difference')
    
    ax.set_xlabel('Time', fontsize=11)
    ax.set_ylabel('PDF', fontsize=11)
    ax.set_title(f'k = {k}', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9, loc='upper right')
    ax.grid(True, alpha=0.3)
    
    # Compute metrics
    mean_dondi = np.trapz(t * pdf_dondi, t)
    mean_explicit = np.trapz(t * pdf_explicit, t)
    max_abs_diff = np.max(np.abs(pdf_dondi - pdf_explicit))
    
    # L2 norm of difference
    l2_diff = np.sqrt(np.trapz((pdf_dondi - pdf_explicit)**2, t))
    
    results_table.append({
        'k': k,
        'mean_dondi': mean_dondi,
        'mean_explicit': mean_explicit,
        'mean_diff': abs(mean_dondi - mean_explicit),
        'max_abs_diff': max_abs_diff,
        'l2_diff': l2_diff
    })
    
    # Annotate with max difference
    ax.text(0.98, 0.75, f'Max diff: {max_abs_diff:.6f}\nL2: {l2_diff:.6f}', 
            transform=ax.transAxes, fontsize=9, 
            verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7))

plt.tight_layout()
plt.suptitle('Dondi vs Explicit Subordination Across k Values', 
             fontsize=14, fontweight='bold', y=1.002)
plt.show()

# Summary table
print("\n" + "="*100)
print("SUMMARY: Dondi vs Explicit Subordination Across k Values")
print("="*100)
print(f"{'k':<8} {'Mean (Dondi)':<15} {'Mean (Explicit)':<18} {'Mean Δ':<12} {'Max |Δ|':<12} {'L2(Δ)':<12}")
print("-"*100)

for result in results_table:
    print(f"{result['k']:<8.1f} {result['mean_dondi']:<15.4f} {result['mean_explicit']:<18.4f} "
          f"{result['mean_diff']:<12.6f} {result['max_abs_diff']:<12.6f} {result['l2_diff']:<12.6f}")

print("="*100)
print("\nOBSERVATION:")
print("- As k → 1 (exponential), differences may approach zero (special case?)")
print("- For k ≠ 1, systematic differences appear")
print("- Question: Which model better matches empirical SEC data?")

### Mathematical Analysis: The Cross Term

Let's examine the **cross term** that appears in Dondi but not in Explicit subordination.

**Dondi formula:**
```
Z = Ψ_stat + iω*t₀
φ_Dondi = exp(Z + Z²/(2N₀))
        = exp(Ψ_stat + iω*t₀ + (Ψ_stat² + 2*iω*t₀*Ψ_stat + (iω*t₀)²)/(2N₀))
```

**Explicit subordination:**
```
φ_Explicit = exp(t₀*Ψ_stat + (t₀²/(2N₀))*Ψ_stat²)
```

**The difference:**
```
φ_Dondi / φ_Explicit = exp(iω*t₀ + (2*iω*t₀*Ψ_stat + (iω*t₀)²)/(2N₀))
                      = exp(iω*t₀ * (1 + Ψ_stat/N₀) - (ω*t₀)²/(2N₀))
```

**Physical interpretation:**

1. **Dondi (cross term present):**
   - Mobile and stationary dispersion are **coupled**
   - The amount of mobile phase dispersion depends on stationary phase retention
   - Physical: Molecules with longer stationary phase time experience more mobile dispersion
   - This is **column-level coupling**

2. **Explicit subordination (no cross term):**
   - Mobile and stationary dispersions are **independent**
   - Subordination is "clean" - just time transformation
   - Physical: Dispersion processes are decoupled
   - This is **mathematical subordination**

**Which is correct?** Depends on the physical system:
- If molecules **retain memory** of stationary residence while in mobile phase → Dondi
- If dispersion processes are **independent** → Explicit
- Empirical validation needed!

In [ ]:
# Visualize the cross term effect
# Let's compute the ratio φ_Dondi / φ_Explicit in frequency domain

k_test = 2.0
theta_test = mean_residence / k_test

# Compute CFs in frequency domain
w_test = np.linspace(-0.5, 0.5, 200)
cf_dondi = dispersive_gamma_pore(w_test, np_, k_test, theta_test, N0, t0_val)
cf_explicit = explicit_subordination_cf(w_test, np_, k_test, theta_test, N0, t0_val)

# Ratio
cf_ratio = cf_dondi / cf_explicit

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Panel 1: Magnitude of CFs
ax = axes[0, 0]
ax.plot(w_test, np.abs(cf_dondi), 'b-', label='|CF_Dondi|', linewidth=2)
ax.plot(w_test, np.abs(cf_explicit), 'r--', label='|CF_Explicit|', linewidth=2)
ax.set_xlabel('Frequency ω', fontsize=11)
ax.set_ylabel('|CF|', fontsize=11)
ax.set_title('Magnitude of Characteristic Functions', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Panel 2: Phase of CFs
ax = axes[0, 1]
ax.plot(w_test, np.angle(cf_dondi), 'b-', label='∠CF_Dondi', linewidth=2)
ax.plot(w_test, np.angle(cf_explicit), 'r--', label='∠CF_Explicit', linewidth=2)
ax.set_xlabel('Frequency ω', fontsize=11)
ax.set_ylabel('Phase (radians)', fontsize=11)
ax.set_title('Phase of Characteristic Functions', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Panel 3: Magnitude of ratio
ax = axes[1, 0]
ax.plot(w_test, np.abs(cf_ratio), 'purple', linewidth=2)
ax.axhline(1, color='black', linestyle=':', alpha=0.5, label='Unity')
ax.set_xlabel('Frequency ω', fontsize=11)
ax.set_ylabel('|CF_Dondi / CF_Explicit|', fontsize=11)
ax.set_title('Ratio Magnitude (shows coupling effect)', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Panel 4: Phase difference
ax = axes[1, 1]
phase_diff = np.angle(cf_ratio)
ax.plot(w_test, phase_diff, 'orange', linewidth=2)
ax.axhline(0, color='black', linestyle=':', alpha=0.5)
ax.set_xlabel('Frequency ω', fontsize=11)
ax.set_ylabel('Phase difference (radians)', fontsize=11)
ax.set_title('Phase Difference (cross term contribution)', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Frequency Domain Analysis: Dondi vs Explicit (k=2.0)', 
             fontsize=14, fontweight='bold', y=1.002)
plt.show()

# Analytical cross term
print("\n" + "="*80)
print("CROSS TERM ANALYSIS")
print("="*80)
print("The ratio φ_Dondi / φ_Explicit = exp(iω*t₀*(1 + Ψ_stat/N₀) - (ω*t₀)²/(2N₀))")
print()
print("At ω=0 (DC component):")
cf_ratio_0 = dispersive_gamma_pore(np.array([0.0]), np_, k_test, theta_test, N0, t0_val)[0] / \
             explicit_subordination_cf(np.array([0.0]), np_, k_test, theta_test, N0, t0_val)[0]
print(f"  Ratio = {cf_ratio_0:.6f} (should be exactly 1)")
print()
print("For small ω:")
print("  Dondi has EXTRA term: exp(iω*t₀*Ψ_stat/N₀)")
print(f"  At ω=0.1: Ψ_stat/N₀ ≈ {np_*(((1-0.1j*theta_test)**(-k_test)-1))/N0:.6f}")
print()
print("Physical meaning:")
print("  - This term couples mobile phase time t₀ with stationary phase CF Ψ_stat")
print("  - Larger N₀ (less mobile dispersion) → weaker coupling")
print("  - Larger Ψ_stat (more stationary retention) → stronger coupling")
print("="*80)

### Summary: Dondi vs Explicit Subordination

**Key Findings:**

1. **Mathematically different:**
   - Dondi includes cross term: `2*iω*t₀*Ψ_stat/(2N₀)`
   - Explicit lacks this coupling
   - Difference grows with retention (larger Ψ_stat)

2. **Physical interpretation:**
   - **Dondi (coupled):** Molecules that spend more time in stationary phase experience more mobile dispersion
   - **Explicit (decoupled):** Subordination is "clean" mathematical transformation

3. **Which is correct?**
   - **For chromatography (Dondi):** Coupling is physical! Longer residence → more axial diffusion
   - **For pure subordination (Explicit):** Mathematical abstraction, assumes independence

4. **Practical implications:**
   - **If N₀ is large:** Cross term → 0, models converge
   - **If N₀ is small:** Coupling effect significant
   - **Your N₀ = 14400:** Moderate coupling, differences visible

5. **Recommendation:**
   - **Use Dondi** for SEC modeling (captures column physics)
   - **Use Explicit** if you want to test pure subordination hypothesis
   - **Validate with simulation:** Which matches empirical data better?

**Next step:** Create validation script comparing both models against SEC simulation data!